In [1]:
# Let's scrape the MAF outputs to get some interesting values
import glob
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pylab as plt
import os
%matplotlib inline


In [2]:
glance_dirs = glob.glob('/Users/yoachim/runs17/technical/maf_new/*_glance')
sci_dirs = glob.glob('/Users/yoachim/runs17/technical/maf_new/*_sci')

glance_dirs.sort()
sci_dirs.sort()

In [7]:
filenames = [name.replace('./', '').replace('_glance', '') for name in glance_dirs]

In [8]:
runNames =  [name.split('/')[-1].replace('_v1.7_10yrs_glance','').replace('v1.7_10yrs_glance','').replace('v1.7_10yrs_glance','').replace('_v1.7.1_10yrs_glance','') for name in glance_dirs] 
versions = ['1.7.1']*len(glance_dirs)

In [9]:
# Set up the dataframe
df = pd.DataFrame(np.array([runNames, versions]).T, columns=['runName', 'version'])

In [10]:
# List of tupes with (metricName, summaryName) that we want to pull from glance dirs
mnamesname =[('parallax', 'best18k'),
            ('properMotion', 'best18k'),
            ('fO', 'fONv MedianNvis'),
            ('fO', 'fOArea')]

for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s") and summaryName= "%s";' % (names[0], names[1])
    for directory in glance_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

In [11]:
# List of tupes with (metricName, summaryName) that we want to pull from science dirs
mnamesname =[('SNNSNMetric_nSN', 'Sum'),
            ('Fast Microlensing', 'Mean'),
            ('Slow Microlensing', 'Mean'),
            ('GalaxyCountsMetric_extended', 'N Galaxies (all)'),
            ('TDEsPopMetric__some_color', 'Mean'),
             ('TDEsPopMetric__some_color_pu', 'Mean')]

In [12]:
for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s") and summaryName= "%s";' % (names[0], names[1])
    for directory in sci_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

In [13]:
sql

'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "TDEsPopMetric__some_color_pu") and summaryName= "Mean";'

In [14]:
df

,runName,version,parallax_best18k,properMotion_best18k,fO_fONv MedianNvis,fO_fOArea,SNNSNMetric_nSN_Sum,Fast Microlensing_Mean,Slow Microlensing_Mean,GalaxyCountsMetric_extended_N Galaxies (all),TDEsPopMetric__some_color_Mean,TDEsPopMetric__some_color_pu_Mean
0,baseline_nexp2,1.7.1,0.785917,0.171124,888.0,17992.777166,30894.342,0.1539,0.9487,1.209168e+10,0.0530,0.0116
1,bulge_roll_scale0.90_nslice2_fpw0.9_nrw1.0,1.7.1,0.780470,0.177617,858.0,16634.800048,37674.050,0.4199,0.9501,1.211450e+10,0.0566,0.0102
2,bulge_roll_scale0.90_nslice3_fpw0.9_nrw1.0,1.7.1,0.787462,0.180220,860.0,16732.158111,38094.834,0.3957,0.9502,1.201279e+10,0.0509,0.0095
3,carina,1.7.1,0.793230,0.172380,884.0,17939.901666,29623.272,0.1878,0.9563,1.202374e+10,0.0534,0.0116
4,footprint_6,1.7.1,0.799966,0.175804,857.0,16109.402226,22330.579,0.4412,0.9486,1.226818e+10,0.0560,0.0116
5,full_disk_scale0.90_nslice2_fpw0.9_nrw1.0,1.7.1,0.756120,0.171740,846.0,15656.183658,38607.762,0.4189,0.9481,1.185737e+10,0.0532,0.0094
6,full_disk_scale0.90_nslice3_fpw0.9_nrw1.0,1.7.1,0.757866,0.174458,848.0,15898.739521,39104.875,0.3883,0.9481,1.174939e+10,0.0513,0.0092
7,full_disk,1.7.1,0.765414,0.169707,848.0,15787.952760,27770.556,0.4400,0.9478,1.201992e+10,0.0534,0.0096
8,rolling_nm_scale0.90_nslice2_fpw0.9_nrw1.0,1.7.1,0.779671,0.175596,889.0,18008.723745,39372.895,0.1485,0.9548,1.177705e+10,0.0569,0.0102
9,rolling_nm_scale0.90_nslice3_fpw0.9_nrw1.0,1.7.1,0.791052,0.179354,892.0,18002.009396,39516.234,0.1644,0.9543,1.166309e+10,0.0512,0.0096


In [16]:
df = df.drop([3])
df

,runName,version,parallax_best18k,properMotion_best18k,fO_fONv MedianNvis,fO_fOArea,SNNSNMetric_nSN_Sum,Fast Microlensing_Mean,Slow Microlensing_Mean,GalaxyCountsMetric_extended_N Galaxies (all),TDEsPopMetric__some_color_Mean,TDEsPopMetric__some_color_pu_Mean
0,baseline_nexp2,1.7.1,0.785917,0.171124,888.0,17992.777166,30894.342,0.1539,0.9487,1.209168e+10,0.0530,0.0116
1,bulge_roll_scale0.90_nslice2_fpw0.9_nrw1.0,1.7.1,0.780470,0.177617,858.0,16634.800048,37674.050,0.4199,0.9501,1.211450e+10,0.0566,0.0102
2,bulge_roll_scale0.90_nslice3_fpw0.9_nrw1.0,1.7.1,0.787462,0.180220,860.0,16732.158111,38094.834,0.3957,0.9502,1.201279e+10,0.0509,0.0095
4,footprint_6,1.7.1,0.799966,0.175804,857.0,16109.402226,22330.579,0.4412,0.9486,1.226818e+10,0.0560,0.0116
5,full_disk_scale0.90_nslice2_fpw0.9_nrw1.0,1.7.1,0.756120,0.171740,846.0,15656.183658,38607.762,0.4189,0.9481,1.185737e+10,0.0532,0.0094
6,full_disk_scale0.90_nslice3_fpw0.9_nrw1.0,1.7.1,0.757866,0.174458,848.0,15898.739521,39104.875,0.3883,0.9481,1.174939e+10,0.0513,0.0092
7,full_disk,1.7.1,0.765414,0.169707,848.0,15787.952760,27770.556,0.4400,0.9478,1.201992e+10,0.0534,0.0096
8,rolling_nm_scale0.90_nslice2_fpw0.9_nrw1.0,1.7.1,0.779671,0.175596,889.0,18008.723745,39372.895,0.1485,0.9548,1.177705e+10,0.0569,0.0102
9,rolling_nm_scale0.90_nslice3_fpw0.9_nrw1.0,1.7.1,0.791052,0.179354,892.0,18002.009396,39516.234,0.1644,0.9543,1.166309e+10,0.0512,0.0096
10,six_stripe_scale0.90_nslice6_fpw0.9_nrw0.0,1.7.1,0.796035,0.192011,864.0,16781.676436,41416.111,0.3353,0.9500,1.183578e+10,0.0401,0.0109


In [17]:
df.to_pickle('metric_df.pkl')